In [1]:
import os

import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, HfArgumentParser,
                          TrainingArguments, logging, pipeline)
from trl import SFTTrainer

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-13 10:04:28,135] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model_name = "llama-2-7b-chat-hf-phr_mental_therapy-2"

In [3]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cpu", ## device map = "cpu", for merging on cpu
)
tokenizer = AutoTokenizer.from_pretrained(new_model_name)
base_model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(base_model, new_model_name)
model = model.merge_and_unload()

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.42it/s]


In [4]:
model.push_to_hub(new_model_name, use_temp_dir=True)
tokenizer.push_to_hub(new_model_name, use_temp_dir=True)

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]



model-00003-of-00003.safetensors:   0%|          | 999k/3.59G [00:02<1:44:03, 575kB/s]    

model-00003-of-00003.safetensors:   0%|          | 3.70M/3.59G [00:03<28:17, 2.11MB/s] 

model-00003-of-00003.safetensors:   0%|          | 4.41M/3.59G [00:03<25:59, 2.30MB/s]

model-00003-of-00003.safetensors:   0%|          | 5.05M/3.59G [00:03<23:51, 2.51MB/s]
model-00003-of-00003.safetensors:   0%|          | 5.49M/3.59G [00:03<22:49, 2.62MB/s]
model-00003-of-00003.safetensors:   0%|          | 10.5M/3.59G [00:04<08:54, 6.69MB/s]
model-00003-of-00003.safetensors:   0%|          | 14.1M/3.59G [00:04<04:44, 12.6MB/s]



model-00003-of-00003.safetensors:   1%|          | 24.1M/3.59G [00:06<08:11, 7.26MB/s]
model-00003-of-00003.safetensors:   1%|          | 24.9M/3.59G [00:06<07:58, 7.45MB/s]
model-00003-of-00003.safetensors:   1%|          | 25.8M/3.59G [00:06<08:21, 7.11MB/s]
model-00003-of-00003.safetensors:   1%

CommitInfo(commit_url='https://huggingface.co/vibhorag101/llama-2-7b-chat-hf-phr_mental_therapy-2/commit/8acf2cbcb08118fbfd9ab4b64af167ab968a766a', commit_message='Upload tokenizer', commit_description='', oid='8acf2cbcb08118fbfd9ab4b64af167ab968a766a', pr_url=None, pr_revision=None, pr_num=None)